In [1]:
import json

# import the smartpynector module
import smartpynector as sp

In [2]:
# Defining constants
# Knowledge engine REST API URL
# KNOWLEDGE_ENGINE_URL = "http://127.0.0.1:8280/rest"
SUBSCRIBER_ID = "http://subscriber:8000/subscriber"
KNOWLEDGE_ENGINE_URL = "http://knowledge_engine:8280/rest"
THERMOSTAT_API_URL = "http://0.0.0.0:8001/thermostat"

In [3]:
# Defining the smart connector
smart_connector = sp.SmartConnector(
    knowledgeBaseId=SUBSCRIBER_ID,
    knowledgeBaseName="Subscriber2",
    knowledgeBaseDescription="Subscriber2",
    reasonerEnabled=False,
)

In [4]:
sp.create_smart_connector(smart_connector, KNOWLEDGE_ENGINE_URL)

2023-05-13 21:07:59 DEBUG Starting new HTTP connection (1): knowledge_engine:8280
2023-05-13 21:08:00 DEBUG http://knowledge_engine:8280 "POST /rest/sc HTTP/1.1" 200 0
2023-05-13 21:08:00 INFO Request successful
2023-05-13 21:08:00 DEBUG 


<Response [200]>

In [5]:
PREFIXES={
        "rdf": "http://www.w3.org/1999/02/22-rdf-syntax-ns#",
        "saref": "https://w3id.org/saref#",
    }
GRAPH_PATTERN= '''?meas rdf:type saref:Measurement .
         ?meas saref:hasValue ?temp .
         ?meas saref:isMeasuredIn saref:TemperatureUnit .
         ?meas saref:hasTimestamp ?timestamp .
         ?meas saref:isMeasurementOf ?room_id .
         ?meas saref:relatesToProperty saref:Temperature .
         ?meas saref:measurementMadeBy ?device_id .'''

In [6]:
ask_ki_id = sp.register_knowledge_interaction(
    knowledge_interaction_type="AskKnowledgeInteraction",
    knowledge_engine_url=KNOWLEDGE_ENGINE_URL,
    prefixes=PREFIXES,
    graph_pattern=GRAPH_PATTERN,
    headers={
        "Content-Type": "application/json",
        "knowledge-Base-Id": SUBSCRIBER_ID,
    },
).json()["knowledgeInteractionId"]
ask_ki_id

2023-05-13 21:08:00 DEBUG Starting new HTTP connection (1): knowledge_engine:8280
2023-05-13 21:08:00 DEBUG http://knowledge_engine:8280 "POST /rest/sc/ki HTTP/1.1" 200 111
2023-05-13 21:08:00 INFO Request successful
2023-05-13 21:08:00 DEBUG {"knowledgeInteractionId":"http://subscriber:8000/subscriber/interaction/e298b07e-a7a9-44db-81c2-a13ad5db7da5"}


'http://subscriber:8000/subscriber/interaction/e298b07e-a7a9-44db-81c2-a13ad5db7da5'

In [7]:
binding_set = sp.perform_ask_knowledge_interaction(
    knowledge_engine_url=KNOWLEDGE_ENGINE_URL,
    knowledge_base_id=SUBSCRIBER_ID,
    knowledge_interaction_id=ask_ki_id,
    ask_data=[]
).json()["bindingSet"]

2023-05-13 21:08:00 DEBUG Starting new HTTP connection (1): knowledge_engine:8280
2023-05-13 21:08:01 DEBUG http://knowledge_engine:8280 "POST /rest/sc/ask HTTP/1.1" 200 879
2023-05-13 21:08:01 INFO Request successful
2023-05-13 21:08:01 DEBUG {"bindingSet":[{"room_id":"\"http://0.0.0.0:8001/thermostat/rooms/1\"","temp":"20","device_id":"\"http://0.0.0.0:8001/thermostat/devices/1\"","meas":"<http://0.0.0.0:8001/thermostat/measurements/af8dc12f-ad2a-49da-a19c-b9f2e261aebf>","timestamp":"\"2023-05-13T21:07:50+00:00\""}],"exchangeInfo":[{"initiator":"knowledgeBase","knowledgeBaseId":"http://0.0.0.0:8001/thermostat","knowledgeInteractionId":"http://0.0.0.0:8001/thermostat/interaction/c946e535-502d-4ccd-978f-c10008ee2fb1","exchangeStart":"2023-05-13T21:08:01.282+00:00","exchangeEnd":"2023-05-13T21:08:01.457+00:00","status":"SUCCEEDED","bindingSet":[{"room_id":"\"http://0.0.0.0:8001/thermostat/rooms/1\"","temp":"20","device_id":"\"http://0.0.0.0:8001/thermostat/devices/1\"","meas":"<http://0

In [8]:
READ_URL = "https://graphdb.odissei.nl/repositories/MateuszTest"
WRITE_URL = "https://graphdb.odissei.nl/repositories/MateuszTest/statements"

sp.store_data_in_graphdb(
    graph_pattern=GRAPH_PATTERN,
    binding_set=binding_set,
    prefixes=PREFIXES,
    read_url=READ_URL,
    write_url=WRITE_URL,
)

2023-05-13 21:08:02 DEBUG Ignoring 500 server error
